# Hlasování

<b>Zdroj dat: https://www.psp.cz/eknih/cdrom/opendata/</b>

Formát dat
Data jsou dostupná ve formátu UNL, tj. každý řádek v souboru odpovídá jednomu řádku v databázi, oddělovačem je znak roury (|), pokud je sloupec prázdný, je jeho hodnota typu null. V sloupcích jsou používány tzv. escape sekvence k zápisu speciálních znaků s úvodním znakem \ (backslash) následovaný znakem.

Kódování je windows-1250. \[edit: až na výjimky\]

Pokud bude strunktura dat doplňována, budou nové sloupce přidávány na konec.

Data jsou poskytována bezplatně, využití dat je <b>podmíněno uvedením zdroje dat a případně datem zpracování dat</b>.


<b>Jmenné konvence:</b>
* tabulky a sloupce tabulek česky dle konvence v zdrojových datech
* proměnné, které se semanticky vztahují k dění v poslanecké sněmovně, česky bez diakritiky nebo anglicky
* ostatní proměnné anglicky
* komentáře a vysvětlení česky s diakritikou
* funkce anglicky

# Parametry

In [ ]:
volebni_obdobi = '2017'
data_dir = './data/'

# Instalace závislostí

In [ ]:
!pip install jupyter-dash -q

In [ ]:
%load_ext autoreload
%autoreload 2

# Importy

In [ ]:
from setup_logger import log

In [ ]:
from os import path

from datetime import timezone, datetime
import pytz

import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

import dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [5]:
from utility import *
from Snemovna import *
from Osoby import *
from Hlasovani import *
from Schuze import *

In [6]:
s = Schuze(volebni_obdobi=2017, data_dir="test7", stahni=True)

test7/schuze.zip
Checking for: ['test7/schuze.unl']
Missing files: []
Tabulka schuze:
Přetypovávám sloupec: 'id_schuze'.
Přetypovávám sloupec: 'id_org'.
Přetypovávám sloupec: 'schuze'.


TypeError: data type "Int" not understood

In [ ]:
s.df

# Tabulka hlasování

In [ ]:
hl = Poslanec(2017, data_dir="data_test/", stahni=True)

## Načtení tabulky hlasování (hlasovani_df)

In [ ]:
hlasovani_df = hl.df

In [ ]:
hlasovani_df.head()

## Analýza tabulky hlasování (hlasovani_df)

In [ ]:
popis_tabulku(hl.df)

In [ ]:
popis_sloupec(hlasovani_df, 'bod')

In [ ]:
# Bod pořadu schůze; je-li menší než 1, pak jde o procedurální hlasování nebo o hlasování k bodům, které v době hlasování neměly přiděleno číslo.
fig = groupby_bar(hlasovani_df, by=['bod_CAT'], xlabel='bod hlasování', ylabel='počet', title="Počet hlasování dle bodu")
fig.show()

In [ ]:
popis_sloupec(hlasovani_df, 'vysledek_CAT')

In [ ]:
fig = groupby_bar(hlasovani_df, by=['vysledek_CAT'], xlabel='výsledek hlasování', ylabel='počet hlasování', title="Počet hlasování dle výsledku")
fig.show()

In [ ]:
popis_sloupec(hlasovani_df, 'druh_hlasovani_CAT')

In [ ]:
fig = groupby_bar(hlasovani_df, by=['druh_hlasovani_CAT'], xlabel='druh hlasování', ylabel='počet hlasování', title="Počet hlasování dle druhu hlasování")
fig.show()

In [ ]:
popis_sloupec(hlasovani_df, 'datetime')

In [ ]:
# hlasovani dle hodiny

df = hlasovani_df.groupby(hlasovani_df.datetime.dt.hour).size()
max_idx = df.sort_values().index[-1]
min_idx = df.sort_values().index[0]
print(f"Nejvíce hlasování ({df.loc[max_idx]}) se uskutečnilo mezi {max_idx}:00 a {max_idx}:59.")
print(f"Nejméně hlasování ({df.loc[min_idx]}) se uskutečnilo mezi {min_idx}:00 a {min_idx}:59.")

fig = go.Figure()

fig.add_trace(go.Bar(
    x=df.index,
    y=df.values,
    text=[f"{t}:00 - {t}:59" for t in df.index],
    hovertemplate="Interval: %{text}<br>Počet hlasování:%{y}<extra></extra>",
))

fig.update_layout(
    title="Počet hlasování dle času [v hodinách]",
    xaxis_title="čas v hodinách",
    yaxis_title="počet hlasování",
    width=600,
    height=400
)

fig.show()

In [ ]:
# hlasovani dle dne z týdne

day_dict = {'Monday':'pondělí', "Tuesday":'úterý', "Wednesday":'středa', "Thursday":'čtvrtek', "Friday":'pátek', "Saturday":'sobota', "Sunday":'neděle'}

df = hlasovani_df.groupby(hlasovani_df.datetime.dt.strftime('%A').map(day_dict)).size()
max_idx = df.sort_values().index[-1]
min_idx = df.sort_values().index[0]
print(f"Nejvíce hlasování ({df.loc[max_idx]}) se uskutečnilo během dne: {max_idx}.")
print(f"Nejméně hlasování ({df.loc[min_idx]}) se uskutečnilo během dne: {min_idx}.")

fig = go.Figure()

index = list(day_dict.values())[:5] # pondělí ... pátek
values = [df.loc[d] for d in index]
text = index
fig.add_trace(go.Bar(
    x=index,
    y=values,
    text=text,
    hovertemplate="Den: %{text}<br>Počet hlasování:%{y}<extra></extra>",
))

fig.update_layout(
    title="Počet hlasování dle dne v týdnu",
    xaxis_title="den",
    yaxis_title="počet hlasování",
    width=600,
    height=400
)

fig.show()

In [ ]:
# Všechna pondělní hlasování
# V pondělí se hlasuje velmi zřídka, zřejmě jen na 1. schůzi

hlasovani_df[hlasovani_df.datetime.dt.strftime("%A") == 'Monday']

In [ ]:
def pocet_hlasovani_dle_data(df, resample_to, resample_str, resample_label):
    frame = df.set_index('datetime').resample(resample_to).size()
    frame = frame.mask(frame == 0, None).dropna()

    max_idx = frame.sort_values().index[-1]
    min_idx = frame.sort_values().index[0]
    print(f"Nejvíce hlasování ({frame.loc[max_idx]}) se uskutečnilo {max_idx.strftime(resample_str)}.")
    print(f"Nejméně hlasování ({frame.loc[min_idx]}) se uskutečnilo {min_idx.strftime(resample_str)}.")

    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=frame.index,
        y=frame.values,
        hovertemplate="%{x}<br>počet hlasování: %{y}<extra></extra>"
    ))
    
    fig.update_layout(
        title=f"Počet hlasování dle data ({resample_label})",
        xaxis_title=f"datum ({resample_label})",
        yaxis_title="počet hlasování",
        width=1200,
        height=500
    )

    dt_all = pd.date_range(start=frame.index[0],end=frame.index[-1])
    dt_obs = [d.strftime(resample_str) for d in frame.index]
    dt_breaks = [d for d in dt_all.strftime(resample_str).tolist() if not d in dt_obs]
    # nezobrazuj data bez hlasování
    #dt_all = pd.date_range(start=df.index[0], end=df.index[-1])
    #dt_obs = [d.strftime(resample_str) for d in df.index]
    #dt_breaks = [d for d in dt_all.strftime(resample_str).tolist() if not d in dt_obs]

    fig.update_xaxes(
        rangebreaks=[dict(values=dt_breaks)]
    )
    
    fig.show()

In [ ]:
pocet_hlasovani_dle_data(hlasovani_df, "D", "%Y-%m-%d", "den")

In [ ]:
pocet_hlasovani_dle_data(hlasovani_df, "M", "%Y-%m", "měsíc")

In [ ]:
# Přidej kontrolní sloupec pro součet hlasování
hlasovani_df['soucet_hlasovani'] =  hlasovani_df.pro + hlasovani_df.proti + hlasovani_df.nehlasoval + hlasovani_df.zdrzel

In [ ]:
def resampled_bar(df, field, interval="D", date_format="%Y-%m-%d", operation='sum', name=None, type='scatter'):
    name = name if name is not None else field    
    #frame = df[field].groupby(df["datetime"].dt.strftime(date_format)).size()
    frame = df.set_index("datetime")[field].resample(interval).agg(operation)
    trace = go.Bar(x=frame.index, y=frame.values, name=name)
    return trace

interval = 'M'
fig = go.Figure()
fig.add_trace(resampled_bar(hlasovani_df, field="proti", interval=interval))
fig.add_trace(resampled_bar(hlasovani_df, field="pro", interval=interval))
fig.add_trace(resampled_bar(hlasovani_df, field="zdrzel", interval=interval))
fig.add_trace(resampled_bar(hlasovani_df, field="nehlasoval", interval=interval))
#fig.add_trace(resampled_bar(hlasovani_df, field="soucet_hlasovani", interval=interval))
fig.add_trace(resampled_bar(hlasovani_df, field="prihlaseno", interval=interval))
fig.add_trace(resampled_bar(hlasovani_df, field="kvorum", interval=interval))

fig.show()

In [ ]:
# Najdi nekonzistentní součty vysledků hlasování, tj. případy, kdy hlasoval jiný počet poslanců, než kolik jich bylo přihlášeno k hlasování.
# TODO: Ověř, že poslanci mají právo hlasovat, i když nejsou přihlášeni.

fields = ["schuze", "bod", "nazev_dlouhy", "pro", "proti", "nehlasoval", "zdrzel", "prihlaseno", "vysledek_CAT", "zmatecne_IND"]
hlasovani_df[hlasovani_df.soucet_hlasovani != hlasovani_df.prihlaseno][fields]

In [ ]:
hlasovani_df["mira_jednomyslnosti"] = (hlasovani_df.pro - hlasovani_df.proti).abs() / (hlasovani_df.pro + hlasovani_df.proti)
hlasovani_df["mira_jednomyslnosti"].describe()

In [ ]:
interval = 'W'
fig = go.Figure()
fig.add_trace(resampled_bar(hlasovani_df, field="mira_jednomyslnosti", interval=interval, operation="mean"))
fig.show()

In [ ]:
# denní míra jednomyslnosti

precision = 3 

def A_razeno_dle_B(x, A, B):
    df = x.groupby(A)[B].mean().sort_values(ascending=True)
    return("<br>".join(map(str, df.index.astype(str) + ': ' + df.values.round(precision).astype(str))))

days = 7
interval = f"{days}D"

df = pd.DataFrame()
df['mira_jednomyslnosti_za_den_mean'] = hlasovani_df.groupby(hlasovani_df.datetime.dt.date).mira_jednomyslnosti.mean()
df['nazev_dlouhy_za_den'] = hlasovani_df.groupby(hlasovani_df.datetime.dt.date)[['nazev_dlouhy', 'mira_jednomyslnosti']].apply(
    lambda x: A_razeno_dle_B(x, "nazev_dlouhy", "mira_jednomyslnosti")
) 
df.index = pd.to_datetime(df.index)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df.index,
    y=df.mira_jednomyslnosti_za_den_mean,
    text=df.nazev_dlouhy_za_den,
    name="míra jednomyslnosti za den",
    mode="markers",
     marker=dict(
        size=5,
        color=df.mira_jednomyslnosti_za_den_mean,
        colorscale='RdYlGn', # one of plotly colorscales
        showscale=True
    ),
    hovertemplate =
        '<b>%{x}</b>'+
        '<br><br><b>Průměrná míra jednomyslnosti</b>: <i>%{y:.3f}</i>'+
        '<br><br><b>Hlasování dle míry jednomyslnosti (vzestupně):</b><br>%{text}<extra></extra>',
    hoverlabel=dict(
        bgcolor='white',
        font=dict(color='black')
    )
))

fig.update_layout(title="Denní míra jednomyslnosti", xaxis=dict(title="datum [jednotlivé dny]"), yaxis=dict(title="míra jednomyslnosti"))

fig.show()

In [ ]:
fig = go.Figure(go.Histogram(x=hlasovani_df.mira_jednomyslnosti, hovertemplate="míra jednomyslnosti: %{x}<br>počet hlasování: %{y}<extra></extra>"))
fig.update_layout(width=700, height=400, xaxis=dict(title="míra jednomyslnosti"), yaxis=dict(title="počet hlasování"))
fig.show()

In [ ]:
print(f"Jednomyslně schválená hlasování (vysoká míra jednomyslnosti): {hlasovani_df[hlasovani_df.mira_jednomyslnosti == 1].index.size}")
jednomyslna_temata = hlasovani_df[hlasovani_df.mira_jednomyslnosti == 1].groupby('nazev_dlouhy').size().sort_values(ascending=False)
print(jednomyslna_temata.head(20))

In [ ]:
print(f"Hlasování se zcela vyrovnaným pro a proti (nízká míra jednomyslnosti): {hlasovani_df[hlasovani_df.mira_jednomyslnosti == 0].index.size}")
nejednomyslna_temata = hlasovani_df[hlasovani_df.mira_jednomyslnosti == 0].groupby('nazev_dlouhy').size().sort_values(ascending=False)
print(nejednomyslna_temata.head(20))

In [ ]:
# Témata hlasování s nejvyšší mírou jednomyslnosti a jejich počty

cnt = 20
x = pd.DataFrame()
x['celkovy_pocet_hlasovani_dle_tematu'] =  hlasovani_df.groupby('nazev_dlouhy').size()
x['prumerna_mira_jednomyslnosti_dle_tematu'] = hlasovani_df.groupby('nazev_dlouhy')['mira_jednomyslnosti'].mean()
x.sort_values(by='prumerna_mira_jednomyslnosti_dle_tematu', ascending=False).head(cnt)

In [ ]:
# Témata hlasování s nízkou mírou jednomyslnosti a jejich počty

cnt = 20
x = pd.DataFrame()
x['celkovy_pocet_hlasovani_dle_tematu'] =  hlasovani_df.groupby('nazev_dlouhy').size()
x['prumerna_mira_jednomyslnosti_dle_tematu'] = hlasovani_df.groupby('nazev_dlouhy')['mira_jednomyslnosti'].mean()
x.sort_values(by='prumerna_mira_jednomyslnosti_dle_tematu', ascending=False).tail(cnt)

## Tabulka stenozáznamů hlasování

In [ ]:
popis_tabulku(hl.stenozaznam)

In [ ]:
hl.stenozaznam.typ_CAT.value_counts()

In [ ]:
# Jak se stenozáznam promítá do tabulky hlasování?

popis_sloupec(hlasovani_df, column= "stenozaznam_IND")
print()

# Hodnoty z tabulky stenozáznam
print("Hodnoty z tabulky stenozáznam - nemusí nutně patřit do aktuální sněmovny:") # TODO: vynech hodnoty, které nepatří do aktuální sněmovny 
hl.stenozaznam.set_index('id_hlasovani').typ_CAT.value_counts()

## Tabulka zpochybnění hlasování (zpochybneni_hlasovani_df)

In [ ]:
zp = ZpochybneniHlasovani(2017)

In [ ]:
zp.df

In [ ]:
# 0 - žádost o opakování hlasování - v tomto případě se o této žádosti neprodleně hlasuje a teprve je-li tato žádost přijata, je hlasování opakováno;
# 1 - pouze sdělení pro stenozáznam, není požadováno opakování hlasování.

zp.df["mode_CAT"].value_counts()

In [ ]:
# TODO: pridej hlášení o neinterpretovatelných sloupcích
#column = 'mode'
#df = zp.df
#h = list(set(df[column].unique()) - set(semanticka_maska.keys()))
#print(f"Neinterpretovatelné hodnoty v sloupci: '{column}': {h}")

In [ ]:
# Jsou zpochybněná hlasování uvedená v tabulce hlasovani_df? 

h_ids = set(hl.df.id_hlasovani)
zp_all_ids = set(zp.df.id_hlasovani)
zp_opak_ids = set(zp.df[zp.df.mode_CAT == "žádost o opakování hlasování"].id_hlasovani)
zp_bez_opak_ids = set(zp.df[zp.df.mode_CAT == "pouze sdělení pro stenozáznam"].id_hlasovani)
print(f"Všechna hlasování: {len(h_ids)}")
print(f"Všechna zpochybněná hlasování = {len(h_ids.intersection(zp_all_ids))}")
print(f"Všechna zpochybněná hlasování s opakováním hlasování = {len(h_ids.intersection(zp_opak_ids))}")
print(f"Všechna zpochybněná hlasování bez opakování hlasování = {len(h_ids.intersection(zp_bez_opak_ids))}")

In [ ]:
# příklad zpochybněných hlasování
fields = ['nazev_dlouhy', 'schuze', 'cislo', 'bod', 'datum', 'cas', 'pro', 'proti', 'zdrzel', 'nehlasoval', 'prihlaseno', 'kvorum', 'vysledek']
hlasovani_df[hlasovani_df.id_hlasovani.isin(zp_all_ids)][fields].head()

In [ ]:
# Nejčastěji zpochybňovaná hlasování
cnt = 10
hlasovani_df[hlasovani_df.id_hlasovani.isin(zp_all_ids)].groupby("nazev_dlouhy").size().sort_values(ascending=False).head(cnt)

## Tabulka zpochybnění hlasování poslancem

In [ ]:
zpp = ZpochybneniPoslancem(2017)

In [ ]:
popis_tabulku(zpp.df)

In [ ]:
zpp.df.head()

In [ ]:
zm = ZmatecneHlasovani(2017)

In [ ]:
zm.df.columns

In [ ]:
hl2013 = Hlasovani(2013)

In [ ]:
# Vizualizace id_hlasovani pro různé tabulky

# Z dat lze usoudit, že pro současnou sněmovnu (2017) neexistuje zmatečné hlasování, které by nebylo uvedeno v tabulce hlasování,
# protože všechny záznamy 'zmatecne_bez_hlasovani_ids' mají ids menší než je začátek ids hlasovani současné směnovny.

# Z dat lze usoudit, že hlasování nemůže být současně zmatečné a zpochybněné poslancem.

# Divné: Pro současnou sněmovnu existují ids zpochybnění hlasování, která jsou nižší než ids v tabulce hlasování.

id_hlasovani_threshold = hl.df.id_hlasovani.min()

hl2017_ids = sorted(list(set(hl.df.id_hlasovani)))
hl2013_ids = sorted(list(set(hl2013.df.id_hlasovani)))
                    
zp_ids = sorted(list(set(zp.df.id_hlasovani)))
zpp_ids = sorted(list(set(zpp.df.id_hlasovani)))
zm_ids = sorted(list(set(zm.df.id_hlasovani)))
                
zpochybneni_bez_poslance_ids = list(set(zp_ids) - set(zpp_ids))
zpochybneni_a_zmatecne_ids = list(set(zp_ids).intersection(zm_ids))
zmatecne_bez_hlasovani_ids = list(set(zm_ids) - set(hl2017_ids))

marker=dict(size=1, line=dict(width=0))
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=hl2017_ids,
    y = [3]*len(hl2017_ids),
    name = "hlasovani_df (současná sněmovna)",
    hovertemplate="hlasovani_df (současná sněmovna)<br>%{x:.0}<extra></extra>",
    mode="markers"
))

fig.add_trace(go.Scatter(
    x=hl2013_ids,
    y = [2.8]*len(hl2017_ids),
    name = "hlasovani_df (sněmovna 2013)",
    hovertemplate="hlasovani_df (sněmovna 2013)<br>%{x:.0}<extra></extra>",
    mode="markers"
))

fig.add_trace(go.Scatter(
    x=zp_ids, 
    y = [2]*len(zp_ids),
    name="zpochybneni_hlasovani_df (současná sněmovna)",
    hovertemplate="zpochybneni_hlasovani_df (současná sněmovna)<br>%{x:.0}<extra></extra>",
    mode="markers"
))

fig.add_trace(go.Scatter(
    x=zpochybneni_bez_poslance_ids,
    y = [1.5]*len(zpochybneni_bez_poslance_ids),
    name="zpochybnění bez asociovaného poslance (současná sněmovna)",
    hovertemplate="zpochybnění bez uvedeného poslance (současná sněmovna)<br>%{x:.0}<extra></extra>",
    mode="markers"
))

fig.add_trace(go.Scatter(
    x=zpp_ids,
    y = [1]*len(zpp_ids),
    name="zpochybneni_poslancem_df (současná sněmovna)",
    hovertemplate="zpochybneni_poslancem_df (současná sněmovna)<br>%{x:.0}<extra></extra>",
    mode="markers"
))


fig.add_trace(go.Scatter(
    x=zm_ids,
    y = [0]*len(zm_ids),
    name="zmatecne_hlasovani_df (všechny sněmovny)",
    hovertemplate="zmatecne_hlasovani_df (všechny sněmovny)<br>%{x:.0}<extra></extra>",
    mode="markers"
))

fig.add_trace(go.Scatter(
    x=zmatecne_bez_hlasovani_ids,
    y = [-0.5]*len(zmatecne_bez_hlasovani_ids),
    name="zmatečné, ale není v tabulce hlasování",
    hovertemplate="zmatečné, ale není v tabulce hlasování<br>%{x:.0}<extra></extra>",
    mode="markers"
))

fig.add_trace(go.Scatter(
    x=[id_hlasovani_threshold]*2,
    y=[-1, 3.5],
    name="hranice záznamů v tabulce hlasování",
    hovertemplate="hranice záznamů v tabulce hlasování<extra></extra>",
    mode="lines",
    line=dict(width=1, dash='dash')
))

fig.update_layout(title="Číselné hodnoty identifikátorů hlasování v různých tabulkách",  yaxis=dict(title='Tabulka', showticklabels=False), xaxis=dict(title='Identifikátor hlasování (jako integer)'))
fig.show()

In [ ]:
# více osob může zpochybnit jedno hlasování, tj. 'id_hlasování' není pro každý řádek unikátní
print(zpp.df.id_hlasovani.nunique(), zpp.df.index.size)
zpp.df.groupby(["id_hlasovani", "jmeno", "prijmeni"]).size().sort_values()

In [ ]:
#zpp.df.groupby('id_hlasovani')["prijmeni"].apply(list).apply(len).sort_values()
pocet_zpochybnujicich_osob_na_hlasovani = zpp.df.groupby('id_hlasovani')["id_osoba"].apply(list).apply(len).sort_values(ascending=False)
x = pocet_zpochybnujicich_osob_na_hlasovani[pocet_zpochybnujicich_osob_na_hlasovani > 1]
res = pd.DataFrame()
if x.count() > 0:
    res = zpp.df[zpp.df.id_hlasovani == x.index[0]]
res

## Tabulka zmatečného hlasování

## Tabulka zarazení osob (osoby_zarazeni_df)

In [ ]:
header = {
    # Identifikátor osoby, viz osoba:id_osoba
    "id_osoba": 'Int64',
    # Identifikátor orgánu či funkce: pokud je zároveň nastaveno zarazeni:cl_funkce == 0, pak id_o odpovídá organy:id_organ, pokud cl_funkce == 1, pak odpovídá funkce:id_funkce.
    "id_of":  'Int64', 
    # Status členství nebo funce: pokud je rovno 0, pak jde o členství, pokud 1, pak jde o funkci.
    "cl_funkce":  'Int64',
    # Zařazení od
    "od_o":   'datetime64[ns]',
    # Zařazení do
    "do_o":   'datetime64[ns]',  
    # Mandát od. Nemusí být vyplněno a pokud je vyplněno, pak určuje datum vzniku mandátu a zarazeni:od_o obsahuje datum volby.
    "od_f":   'datetime64[ns]',
    # Mandát do. Nemusí být vyplněno a pokud je vyplněno, určuje datum konce mandátu a zarazeni:do_o obsahuje datum ukončení zařazení.
    "do_f":   'datetime64[ns]' 
}

osoby_zarazeni_df = pd.read_csv(osoby_zarazeni_path, sep="|", names = header.keys(), index_col=False, encoding='cp1250')

# načti datová pole a oprav případné chyby
df = osoby_zarazeni_df
correct_dict = {'0205-06-09 00': '2005-06-09 00'} # slovnik manuálních oprav pro preklepy
date_strings = {"od_o": '%Y-%m-%d %H', "do_o": '%Y-%m-%d %H', "od_f": "%d.%m.%Y", "do_f": "%d.%m.%Y" }
for col in date_strings.keys():
    new_col = pd.to_datetime(df[col], format=date_strings[col], errors='coerce')
    errors = df.loc[new_col.isnull()][col].unique().tolist()
    df[col] = new_col.fillna(pd.to_datetime(df[col].map(correct_dict)))

osoby_zarazeni_df = osoby_zarazeni_df.astype(header)

In [ ]:
popis_tabulku(osoby_zarazeni_df)

In [ ]:
zarazeni_osoby_df = pd.merge(osoby_zarazeni_df, osoby_df,  on='id_osoba')
#zarazeni_osoby_df = pd.merge(osoby_zarazeni_df, organy_df,  on='id_osoba') # pripoj i organy
zarazeni_osoby_df.groupby(["id_osoba", "jmeno", "prijmeni"]).size().sort_values(ascending=False)

## Tabulka poslanců (poslanci_df)

In [ ]:
header = { "id_poslanec": 'Int64', "id_osoba":  'Int64', "id_kraj":  'Int64', "id_kandidatka":  'Int64', "id_obdobi":  'Int64',
    "web": 'string', "ulice": 'string', "obec": 'string', "psc": 'string',
    "email": 'string', "telefon": 'string', "fax": 'string', "psp_telefon": 'string', "facebook": 'string',
    "foto":  'Int64'
}
 
poslanci_df = pd.read_csv(poslanec_path, sep="|", names = header.keys(), index_col=False, encoding='cp1250')
poslanci_df = poslanci_df.astype(header)
poslanci_df = pd.merge(poslanci_df, osoby_df,  on='id_osoba')
#poslanci_df = pd.merge(poslanci_df, osoby_zarazeni_df,  on='id_osoba')

posledni_volebni_obdobi = poslanci_df.id_obdobi.max()
poslanci_df = poslanci_df[poslanci_df.id_obdobi == posledni_volebni_obdobi]

print(f"Velikost tabulky poslanců pro poslední volební období: {len(poslanci_df)}")
poslanci_df.head()

## Tabulka omluv ze schůzí PS (omluvy_df)

In [ ]:
# načti tabulku omluv
header = [ "id_organ", "id_poslanec", "den", "od", "do"]

omluvy_df = pd.read_csv(omluvy_path, sep="|", names = header,  index_col=False, encoding='cp1250')
omluvy_df = pd.merge(omluvy_df, poslanci_df, on='id_poslanec')
omluvy_df = omluvy_df[omluvy_df.id_obdobi==posledni_volebni_obdobi]

omluvy_df.head()

In [ ]:
cnt = 20
omluvy_na_poslance = omluvy_df.groupby(["id_poslanec", "jmeno", "prijmeni"]).size().sort_values(ascending=False)
omluvy_na_poslance.head(cnt)

In [ ]:
cnt = 20
omluvy_na_poslance.tail(cnt)

In [ ]:
omluvy_na_poslance.describe()

## Check for Nans

## Analyze Categorical Data

In [ ]:
pocet_schuzi = hlasovani_df.schuze.max()

print(f"Počet schůzí: {pocet_schuzi}.")

In [ ]:
go.Figure(go.Box(y=hlasovani_df.groupby("schuze").size(), name="počet hlasování na schůzi"))

In [ ]:
hlasovani_dle_bodu_srs = hlasovani_df[hlasovani_df.bod >= 1].groupby("nazev_dlouhy").size().sort_values(ascending=False)
cnt = 20
y=hlasovani_dle_bodu_srs[:cnt].index[::-1]
x=hlasovani_dle_bodu_srs[:cnt].values[::-1]
px.bar(x=x, y=y, orientation='h')

In [ ]:
def pocet_dni_na_schuzi(id_schuze):
    return len(hlasovani_df[hlasovani_df.schuze == id_schuze].groupby('datum').size().index)

cnt = 10
schuze_dle_poctu_hlasovani = hlasovani_df.groupby("schuze").count().id_hlasovani.sort_values(ascending=False)
schuze_dle_poctu_hlasovani_str = [f"Během {schuze}. schůze ({pocet_dni_na_schuzi(schuze)} dní) proběhlo {pocet_hlasovani} hlasování, průměrně {pocet_hlasovani/pocet_dni_na_schuzi(schuze):.0f} hlasování za den." 
    for (schuze, pocet_hlasovani) in schuze_dle_poctu_hlasovani.iteritems()]
x = "\n".join(schuze_dle_poctu_hlasovani_str[:cnt])

print(f"Prvních {cnt} schůzí s největším počtem hlasování:\n{x}")

In [ ]:
id_schuze_s_nejvetsim_poctem_hlasovani = schuze_dle_poctu_hlasovani.index[0]
#id_schuze_s_nejvetsim_poctem_hlasovani.groupby('datum')
#pocet_dni = len(hlasovani_df[hlasovani_df.schuze == id_schuze_s_nejvetsim_poctem_hlasovani].groupby('datum').count().index)
                
print(f"Největší počet hlasování ({schuze_dle_poctu_hlasovani.iloc[0]}) proběhl během {id_schuze_s_nejvetsim_poctem_hlasovani}. schůze ({pocet_dni_na_schuzi(id_schuze_s_nejvetsim_poctem_hlasovani)} dní).")
d = hlasovani_df[(hlasovani_df.schuze == id_schuze_s_nejvetsim_poctem_hlasovani) & (hlasovani_df.bod >= 1)].groupby("nazev_dlouhy").size().sort_values(ascending=False)

d

In [ ]:
x = 100 * len(hlasovani_df[hlasovani_df.bod < 1]) / len(hlasovani_df)

print(f"{x:.2f}% z hlasování bylo procedurální nebo nemělo přiděleno číslo.")

In [ ]:
hlasovani_df[hlasovani_df.prihlaseno < hlasovani_df.kvorum]

In [ ]:
hlasovani_df.groupby('druh_hlasovani').size()

In [ ]:
# Počet hlasování na schůzi

s = hlasovani_df.groupby('schuze').size().sort_values()
y = s.index
x = s

go.Figure(go.Histogram(x=x, nbinsx=60))

In [ ]:
# Počet hlasování na den

s = hlasovani_df.groupby('datum').size().sort_values()
y = s.index
x = s

go.Figure(go.Histogram(x=x, nbinsx=60))

In [ ]:
hlasovani_df["pocet_dni_na_schuzi"] = hlasovani_df.groupby(["schuze"]).datum.transform('nunique')

hlasovani_df["den_schuze"] = hlasovani_df.groupby(["schuze", "datum"]).ngroup()
hlasovani_df["den_schuze_min"] = hlasovani_df.groupby(["schuze"]).den_schuze.transform(min)
hlasovani_df["den_schuze_rank"] = (hlasovani_df["den_schuze"] - hlasovani_df["den_schuze_min"] + 1)

x1 = hlasovani_df.groupby(["schuze", "datum", "den_schuze_rank"]).size()
z1 = x1.reset_index(name="pocet_hlasovani")
z2 = x1.reset_index(name="pocet_hlasovani").set_index("datum")

print(f"Na schůzích {sorted(set(range(z2.schuze.max() + 1)) - set(z2.schuze.unique()) - set([0]))} se nehlasovalo.")
px.bar(z1, x="schuze", y="pocet_hlasovani", color="den_schuze_rank", title="Long-Form Input")

In [ ]:
print(f"Poslední běh notebooku: {datetime.now().strftime('%d.%m.%Y %H:%M:%S')}.")